## Тема: Снижение размерности данных

### 1. Обучить любую модель классификации на датасете IRIS до применения PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from sklearn.linear_model import LogisticRegression

In [10]:
# Загрузим игрушечный датасет из sklearn
X, y = load_iris(return_X_y=True)
X.shape

(150, 4)

**Модель до применения PCA**

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=1,
                                                    stratify=y)

In [12]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, y_train) * 100, 2)
acc_log

96.67

**PCA**

In [13]:
# Найдем собственные векторы и собственные значения
covariance_matrix = X.T @ X

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('Собственные значения и собственные векторы в порядке убывания:')
for i in eig_pairs:
    print(i)

Собственные значения и собственные векторы в порядке убывания:
(9208.305070314855, array([0.75110816, 0.38008617, 0.51300886, 0.16790754]))
(315.4543165767577, array([ 0.2841749 ,  0.5467445 , -0.70866455, -0.34367081]))
(11.978042904907412, array([ 0.50215472, -0.67524332, -0.05916621, -0.53701625]))
(3.552570203480046, array([ 0.32081425, -0.31725607, -0.48074507,  0.75187165]))


In [14]:
# Оценим долю дисперсии, которая описывается найденными компонентами.
eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}')

# а теперь оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описываемая каждой из компонент 
[96.5302980653157, 3.3068951313646795, 0.12556535030287802, 0.037241453016734434]
Кумулятивная доля дисперсии по компонентам 
[ 96.53029807  99.8371932   99.96275855 100.        ]


In [15]:
# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack([eig_pairs[i][1].reshape(4,1) for i in range(2)])

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.75110816  0.2841749 ]
 [ 0.38008617  0.5467445 ]
 [ 0.51300886 -0.70866455]
 [ 0.16790754 -0.34367081]]


In [16]:
# Сформируем новую матрицу "объекты-признаки"
Z = X.dot(W)

**Модель после применения PCA**

In [18]:
Z_train, Z_test, y_train, y_test = train_test_split(Z, y,
                                                    test_size=0.2,
                                                    random_state=1,
                                                    stratify=y)

In [19]:
logreg = LogisticRegression()
logreg.fit(Z_train, y_train)
y_pred = logreg.predict(Z_test)
acc_log = round(logreg.score(Z_train, y_train) * 100, 2)
acc_log

95.83

**Вывод:** После применения PCA точность алгоритма упала.

## 2. *Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции [numpy.linalg.svd()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html)